# Using Python for Automating Google Slides and Watermarking

In [ ]:
# Adding in imports here.

import IPython
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import argparse
import json
import requests
import pandas as pd
import oauth2client
import httplib2
import logging
import uuid
import io


In [ ]:
SCOPES = ["https://www.googleapis.com/auth/drive","https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive.readonly","https://www.googleapis.com/auth/presentations","https://www.googleapis.com/auth/presentations.readonly","https://www.googleapis.com/auth/spreadsheets","https://www.googleapis.com/auth/spreadsheets.readonly"]



## Introduction

This is a basic example of using the Slides API.  I have made a copy of the presentation you provided to use as an example. I'll start by showing the basic usage of the Slides API first before I get into creating and manipulating elements.

## Steps

1. We will need to authenticate to Google to use the API.  I am using the OAuth Flow for this. First step is to check if a token already exists.

2. If there are no credentials availible, then we will log in as the user to generate one.

3. Creds are saved in a encrypted and secure format so they will not be requested again on the next run (This will last until the token refresh expires.)

4. Once we have authenticated successfully, we can use the Presentation ID to enumerate the slides and then go through each one to get a count of the Page Elements. (This is for a test.  For the actual run, we will be adding a Page Element with the watermark to the desihnated locations.)

In [ ]:
def authSlides():
    
    # Step 1

    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    
   
   # Step 2

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow= InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        
        # Step 3

        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
        
    service = build('slides', 'v1', credentials=creds)
    
    return service


In [ ]:
service = authSlides()

def getSlides(service, PRESENTATION_ID):
    presentation = service.presentations().get(presentationId=PRESENTATION_ID).execute()
    slides = presentation.get('slides')
    
    # Step 4

    print(f'The presentation contains {len(slides)} slides:')
    for i, slide in enumerate(slides):
        try:
            print(f'- Slide {i + 1} contains {len(slide.get("pageElements"))} Page Elements')
        except:
            print(f'- Slide {i+1} contains no Page Elements')

## Creating Presentations and Slide

The API can also be used to create presentations.  This is something that can be useful if you are wanting to preserve the original presentations while generating new ones with watermarks or other changes added.

### Creating the Presentation Request

In [ ]:
body = {
    "title" : "Example Presentation"
}

presentation2 = service.presentations().create(body=body).execute()

print(f'Created presentation with ID: {presentation2.get("presentationId")}')

### Creating Slides

For this next example, I am going to use the presentation that I used above to add in a slide using a template.

In [ ]:
page_id = str(uuid.uuid1())

request1 = [
    {
        'createSlide': {
            'objectId' : page_id,
            'insertionIndex': '1',
            'slideLayoutReference' : {
                'predefinedLayout' : 'TITLE_AND_TWO_COLUMNS'
            }
        }
    }
]

body = {
    'requests': request1
}

response1 = service.presentations().batchUpdate(presentationId='1PdLWKRpxbPERGcQ6HyXnbSpR_uPE5hV8nP4jUkIQ6xk', body=body).execute()

create_slide_response = response1.get('replies')[0].get('createSlide')
objID = create_slide_response.get('objectId')
print(f'Created slide with ID: {objID}')

### Adding In Text and Shapes

Now that we have the general examples out of the way, we will get into what this project is all about, adding in the text and shapes for the watermark.  I will use the slide that we just created as an example first.





In [ ]:
element_id = 'Watermark_Example_01'
pt350X = {
    'magnitude' : 100,
    'unit' : 'PT'
}

pt350Y = {
    'magnitude' : 100,
    'unit' : 'PT'
}


requests2 = [
    {
        'createShape': {
            'objectId': element_id,
            'shapeType': 'TEXT_BOX',
            'elementProperties': {
                'pageObjectId': page_id,
                'size': {
                    'height': pt350X,
                    'width': pt350Y
                },
                'transform': {
                    'scaleX': 3,
                    'scaleY': 0.25,
                    'translateX': 245,
                    'translateY': 365,
                    'unit': 'PT'
                }
            }
        }
    },

    {
        'insertText': {
            'objectId' : element_id,
            'insertionIndex' : 0,
            'text': 'The following information is classified'
        }
    }
]


body = {
    'requests': requests2
}

response2 = service.presentations().batchUpdate(presentationId='1PdLWKRpxbPERGcQ6HyXnbSpR_uPE5hV8nP4jUkIQ6xk', body=body).execute()
create_shape_response = response2.get('replies')[0].get('createShape')
objID2 = create_shape_response.get('objectId')
print(f'Created textbox with ID: {objID2}')

### Adding in Images

Now, text boxes may work for some situations, there may be the need to have an image that is added instead.  This will follow a similar process and transforms can also be added to change things like the scale and positioning of the image that is being added.

## Performing The Export


In [ ]:
def file_search():
    with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    drive_service = build('drive', 'v3', credentials=creds)
    
    results = []
    page_token = None
    while True:
        # response = drive_service.files().list(q="modifiedTime > '2020-11-04T12:00:00'").execute()
        response = drive_service.files().list(q="name='Copy of Splunk Product Line - IT Roadmap - FY21Q3'",fields='nextPageToken, files(id, name,mimeType)', pageToken=page_token).execute()
        for file in response.get('files', []):
            #print(f'Found:  {file.get("name")}, ID: {file.get("id")}')
    
            file_name = file.get("name")
            file_id = file.get("id")
            file_mimeType = file.get("mimeType")
            res = {
                "name": file_name,
                "id" : file_id,
                "mimeType" : file_mimeType
            }
            results.append(res)
        page_token = response.get('nextPageToken', None)
        if page_token is None:
                break
    return results
    

        
        


In [ ]:
files = file_search()

In [ ]:
print(files)

In [ ]:
#@title Exporting the file to PDF and uploading to GDrive
def exportPDF(file_id,filename):

    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
    drive_service = build('drive', 'v3', credentials=creds)
    request3 = drive_service.files().export_media(fileId=file_id,mimeType='application/pdf')
    filename += '.pdf'
    
    fh = io.FileIO(filename, 'wb')
    from googleapiclient.http import MediaIoBaseDownload, MediaDownloadProgress, MediaFileUpload
    downloader = MediaIoBaseDownload(fh, request3, 1024 * 1024 * 1024)
    done = False
    while done is False:
        try:
            status, done = downloader.next_chunk()
            print("Starting Download")
            sys.stdout.flush()
        except Exception as e:
            print(e.message)
            print("Download failed")
            fh.close()
            sys.exit(1)
    print("Complete")
    
    metadata = {'name': filename,'mimeType': 'application/pdf'}
    media = MediaFileUpload(filename,mimetype='application/pdf', chunksize = 1024 * 1024)
    request4 = drive_service.files().create(body=metadata, media_body=media).execute()
    print(request4)



In [ ]:


    exportPDF(file_id,filename)


In [ ]:
def export2():

    from PyDrive.auth import GoogleAuth
    from PyDrive.drive import GoogleDrive

    gauth = GoogleAuth()
    gauth.LocalWebServerAuth()

    drive = GoogleDrive(gauth)

    file_list = drive.ListFile({'q':"name='Example Presentation'"}).GetList()
    for f in file_list:
        print(f'title {f["title"]}, ID: {f["id"]}')